## WATEM :: WhatsAppTEM

### WhatsApp XPATH in Global Vars
whatsapp has 2 section
1. sidepane
2. body

### Library Imports

In [25]:
import os, sys, time, subprocess
from collections import defaultdict
import datetime as dt
from dateutil.parser import *
import time as tm
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select


mypath = os.path.dirname(os.path.realpath('.')) #'__file__'
sys.path.insert(0,mypath)
#from whatsapp import *

### Developed Scrap Functions 

In [26]:
def xc(driver, xpt, click=True): #clickable
    try:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.XPATH, xpt)))
        if click: element.click()
        else: return element
    except: return False    

def xfill(driver, xpt, txt=None, search_only =True):
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.XPATH, xpt))) if type(xpt) is str else xpt
    if element != False:
        element.clear()
        ac = ActionChains(driver)
        ac.move_to_element(element).click().perform()
        if txt is not None and chr(10) in txt and search_only==False:
            xx = txt.split(chr(10))
            for line in xx:
                ac.send_keys(line).perform()
                ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
            ac.send_keys(Keys.RETURN).perform()
        elif txt is not None and search_only==False:
            ac.send_keys(name).send_keys(Keys.RETURN).perform()
        elif txt is not None and search_only==True:
            ac.send_keys(name).perform()

### MSG-Content Scraper (with editable xpath)

In [27]:
def bs_div_attr(html, search_attr='data-pre-plain-text',tag="div"):
    sp = BeautifulSoup(html, "lxml")
    x = sp.find_all(tag)
    for i in x:
        try:
            st = i[search_attr]
            break
        except:pass
    else: return None, None
    try:
        split_point = st.find(']')
        sender = st[split_point+1:].replace(':','').strip(' ')
        dttm = parse(st[1:split_point].upper())
        return dttm.strftime("%Y-%m-%d %H:%M"), sender
    except: None, None
        
def xpob(driver, xpt):
    wait = WebDriverWait(driver, 2)
    try: return wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
    except: return False
    
def msg_scrap(driver, n=4, base="(//div[@id='main']//div[@class='ItfyB _3nbHh'])"):
    msgdiv = base + "[" + str(n) + "]"
    try: ActionChains(driver).move_to_element(xpob(driver, msgdiv)).perform()
    except: pass
    dt_plus_sender_xpt = msgdiv + "//div[@class='_27K43 _31p5Q']"
    sender_msg_xpt = msgdiv + "//div[@class='_21Ahp']/span[1]/span"
    sender_msg_xpt_1 = msgdiv + "//div[@class='_27K43 _31p5Q']//span/span"
    quoted_sender_xpt = msgdiv + "//div[@class='_3pMOs yKTUI']//div[1]/span"
    quoted_text_xpt = msgdiv + "//div[@class='_3pMOs yKTUI']//div[2]/span"
    read_more_button = msgdiv + '/div[1]/div[2]/div/div'
    rd = xpob(d, read_more_button)
    if type(rd) is not bool:
        if 'more' in rd.text.lower() or 'Read' in rd.text:
            try: ActionChains(driver).move_to_element(rd).click().perform()
            except: pass
            xc(driver, read_more_button)
    
    dttm,sender,sender_msg,quoted_sender,quoted_text = '','','','',''
    a = xpob(driver, dt_plus_sender_xpt)
    if type(a) is bool: return None
    
    b = a.get_attribute('innerHTML')
    dttm, sender = bs_div_attr(b)
    try:
        sender_msg = xpob(driver, sender_msg_xpt).text
    except:
        sender_msg = xpob(driver, sender_msg_xpt_1).text
    if type(xpob(d, quoted_sender_xpt)) is not bool:
        quoted_sender = xpob(driver, quoted_sender_xpt).text
        quoted_text = xpob(driver, quoted_text_xpt).text
    return [dttm,sender,sender_msg,quoted_sender,quoted_text]

In [28]:
def sidepane_scroll_by_id(driver, initial = 1, scrolls=25):
    for i in range(0, scrolls):
        driver.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
        initial += 300
    return initial

def scroll_by_xpath(driver, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]"):
    wait = WebDriverWait(driver, 30)
    driver.execute_script("arguments[0].scrollIntoView(true);", wait.until(EC.presence_of_element_located((By.XPATH, xpt))))
    return 1
    
def reload(driver, wait_sec=60):
    wait = WebDriverWait(driver, wait_sec)
    driver.refresh()
    try:
        driver.switch_to.alert.accept()
    except:
        pass
    search_bar_xpt = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
    element = wait.until(EC.presence_of_element_located((By.XPATH, search_bar_xpt)))
    print('reloaded success')

### Director Class

In [29]:

class Wakit:
    
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.wait = WebDriverWait(self.driver, 5)
        self.ac = ActionChains(self.driver)
        self.cdata = defaultdict()
        self.parsed_counter = 0
        self.store_msg = pd.DataFrame(columns=['dttm','sender','sender_msg','quoted_sender','quoted_text','insert_dttm','chat_name'])
        print(self.driver.title)
        
    def isPresent(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        except: return 0
    
    def FetchText(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt))).text
        except: return 0
        
    def select_chat(self, name):
        xpt = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
        element = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        element.clear()
        #self.ac.send_keys(name).send_keys(Keys.RETURN).perform()
        self.ac.move_to_element(element).click().perform()
        self.ac.send_keys(name).send_keys(Keys.RETURN).perform()
        #self.ac.send_keys(name).send_keys(Keys.RETURN).perform()
        return 1
        
    def chain(self, xptdict):
        ''' following xpdict argument open chat name 'onami' and send message 'HI'
            xptdict = {"//div[@class='_2vDPL']" : {'cmd':'paste', 'text':'onami'},
             "//footer[@class='_3E8Fg']//p": {'cmd':'paste','text':'HI'}}
        '''
        for xp, v in xptdict.items():
            x1 = self.isPresent(xp)
            if x1 != 0: 
                self.chain_1(x1, v)
            else:
                print(xp, ' Not Found')

    def chain_1(self, x1,  v):
        if 'click' in v['cmd']:
            self.ac.move_to_element(x1).click().perform()
            print('clicked')
        elif 'paste' in v['cmd']:
            if v['text'] is not None and chr(10) in v['text']:
                self.write_multiline(v['text'])
            elif v['text'] is not None:
                self.ac.move_to_element(x1).click().perform()
                if 'paste-only' in v['cmd']:
                    self.ac.send_keys(v['text']).perform()
                else:
                    self.ac.send_keys(v['text']).send_keys(Keys.RETURN).perform()
    
    def write_long_msg(self, x1, text):
        x1 = isPresent(x1) if type(x1) is str else x1
        self.ac.move_to_element(x1).click().perform()
        xx = text.split(chr(10))
        for line in xx:
            self.ac.send_keys(line).perform()
            self.ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
        else: 
            self.ac.send_keys(Keys.RETURN).perform()
            
    def storing_msg(self, ls):
        if ls is not None and len(ls)>1:
            self.store_msg.loc[len(self.store_msg.index)] = ls
    
    def getxpt(self, xpt):
        wait = WebDriverWait(self.driver, 3)
        try: return wait.until(EC.presence_of_element_located((By.XPATH, ''.join(xpt)))).text
        except: 0
    
    def message_scraper(self, chat_name='emergency', msg_container=30):
        insert_dttm = dt.datetime.now().strftime('%Y-%m-%d %H:%M')
        if self.select_chat(chat_name) == 1: pass
        else: return chat_name + " selection problem"
        base = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])"
        for i in range(1, msg_container):
            bs = base + '[' + str(msg_container) + ']'
            while self.getxpt(bs) == 0 or self.getxpt(bs) == None or self.getxpt(bs) == '':
                scroll_by_xpath(self.driver, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[1]")
                tm.sleep(0.5)
                try: self.ac.move_to_element(isPresent("(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]"))
                except: pass
                bs = base + '[' + str(msg_container) + ']'
            print('container: ', i)
            m = msg_scrap(self.driver, i)
            mx = m + [insert_dttm , chat_name] if m is not None else m
            print(mx)
            self.storing_msg(mx)
        else:
            self.store_msg.to_csv(chat_name + dt.datetime.now().strftime('%d-%M-%Y_%H%M') + '.csv')
    
    @property
    def get_driver(self):
        return self.driver
    
    def clkcross(self, cross_search_bar = ["//div[@class='_2vDPL']",]):
        for ii in cross_search_bar:
            try:
                element = self.wait.until(EC.element_to_be_clickable((By.XPATH, ii)))
                self.ac.move_to_element(element).click().perform()
            except:
                pass

In [31]:
x = Wakit()
d = x.get_driver
x.message_scraper('emergency', 20)

(10) WhatsApp
container:  1
['2023-01-20 19:04', 'Hosna Ara', 'Update of Site Down at 19:00 On 20-01-2023\n\nRegion: 2G/3G/4G\nNational: 23/7/10\n\nDHK_M: 4/0/1\nDHK_N: 0/0/0\nDHK_S: 2/0/0\nDHK_O: 5/0/0\nCTG_M: 2/1/1\nCTG_N: 1/0/0\nCTG_S: 0/0/0\nCOM: 1/0/0\nNOA: 5/0/5\nSYL: 0/0/0\nMYM: 1/1/1\nBAR: 0/1/0\nKHL: 2/4/2\nKUS: 0/0/0\nRAJ: 0/0/0\nRANG: 0/0/0', '', '', '2023-01-21 05:09', 'emergency']
container:  2
['2023-01-20 19:30', 'tawhid vai', 'Net Status 19-Jan,2023\n\nVLR(A): 43.45M(+39.53K)\nVLR(R): 45.15M(+1.35K)\nHLR: 112.22M(+0.9K)\n4G(A): 18.71M(-31.94K)\n4G(R): 24.44M(+53.27K)\nEL(WD): 5.93M(+747.74K)\nELSR: 98.77%(+1.03%)\n\nNetwork Performance:\nCSSR2G: 99.24%(+0.13%)\nCDR2G: 0.36%(-0.03%)\nCDR3G: 0.19%(+0%)\nPSR: 94.91%(+3.55%)\nLUSR: 98.84%(+0.38%)\nMTCCR: 92.6%(-0.33%)\nSMMTSR: 59.74%(+0.78%)\n\nTraffic:\nVoice:\n2G(BH): 457.67K(-35.79K)\n2G(WD): 5050.97K(-74.79K)\n3G(BH): 269.36K(-12.48K)\n3G(WD): 3119.59K(-42.88K)\n\nData:\nGiDL(TP): 1164.5Gbps(-4.7Gbps)\nGiDL(FNA): 494Gbp

In [22]:
x.select_chat('emergency')

1

NameError: name 'Keys' is not defined

In [8]:
cross_search_bar = ['/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/span/button/span',
                    '/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/button/div[2]/span']
for ii in cross_search_bar:
    try:
        element = self.wait.until(EC.element_to_be_clickable((By.XPATH, ii)))
        self.ac.move_to_element(element).click().perform()
    except:
        pass

(25) WhatsApp


'emergency selection problem'

In [34]:
read_more_button = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]" + '/div[1]/div[2]/div/div'
rd = xpob(d, read_more_button)
if type(rd) is not bool:
    if 'more' in rd.text.lower():
        ac = ActionChains(d)
        ac.move_to_element(rd).click().perform()
        tm.sleep(1)

In [ ]:
def bodymsg(nodeno=2, bodybase = "(//div[@class='ItfyB _3nbHh'])"):
    base = bodybase + "[" + str(nodeno) + ']'
    bodyxpt = {'sender':[base + '/div/div[1]'],
               'sender_text':[ base + '/div/div[2]'],
               'send_time':[base + '/div/div[3]'],
               'Q_Clickable':[base + '/div[1]/div/div/div/div'],
               'Q_Sender':[base + '//div[2]/div[1]/div/div/div/div/div[1]/span'],
               'Q_Text':[base + '//div[2]/div[1]/div/div/div/div/div[2]/span']}
    return bodyxpt

def msgtext(div_number=2,base="(//div[@id='main']//div[@class='ItfyB _3nbHh'])"):
    b = base + '[' + str(div_number) + ']'
    msgxpt = {'datetime': b + "//div[@class='_27K43 _31p5Q']/div",
              'msg_sender' : b + "//span[@class='_3FuDI ajgl1lbb edeob0r2 _11JPr']",
              'msg_text' : b + "//div[@class='_21Ahp']//span[@class='_11JPr selectable-text copyable-text']",
              'Quote_Sender' : b + "//div[@class='_1hl2r']//span[@class='_3FuDI _11JPr']",
              'Quote_Text' : b + "//div[@class='_1hl2r']//div[@class='_37DQv']"}
    return msgxpt

def getxpt(driver, xpt):
    wait = WebDriverWait(driver, 3)
    try: return wait.until(EC.presence_of_element_located((By.XPATH, ''.join(xpt)))).text
    except: 0

def iter_msg_container():
    base = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])"
    for i in range(30, 2, -1):
        bs = base + '[' + str(i) + ']'
        while getxpt(d, bs) == 0 or getxpt(d, bs) == None or getxpt(d, bs) == '':
            scroll_by_xpath(d, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[1]")
            tm.sleep(3)
        x = bodymsg(i, base)
        for k, v in x.items():
            print(k, ': ',getxpt(d, v))
            print('---------------------')
        print('##########################################')

In [ ]:
iter_msg_container()

In [ ]:
msg_arrow =lambda x : "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[" + str(x) + "]//div[@class='_3u9t-']/span"
msg_reply = lambda x: "//ul[@class='_3bcLp']/div/li[@class='Iaqxu FCS6Q jScby'][1]"
msg_forward = lambda x: "//ul[@class='_3bcLp']/div/li[@class='Iaqxu FCS6Q jScby'][4]"
msg_delete = lambda x: "//ul[@class='_3bcLp']/div/li[@class='Iaqxu FCS6Q jScby'][7]"
msg_react = lambda x: "//ul[@class='_3bcLp']/div/li[@class='Iaqxu FCS6Q jScby'][3]"
msg_react_plus_btn = "/html/body/div[1]/div/span[4]/div/div/div/div/div/div/div[7]/div/span"

In [ ]:
//*[contains(text(),'match')]

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['Col1', 'Col2', 'Col3'])

In [ ]:
df.loc[len(df.index)] = ['CMSDR05', '3G', 'P2']
df

In [ ]:
df

In [ ]:
df.loc[len(df.index)] = ['CMSDR05', '3G', 'P2']

In [ ]:
df

In [ ]:
# tested sucess
# following xpdict argument open chat name 'onami' and send message 'HI'
#c1 = {"//div[@class='_2vDPL']" : {'cmd':'paste', 'text':'onami'},
#     "//footer[@class='_3E8Fg']//p": {'cmd':'paste','text':'HI'}}
#x.chain(c1)

In [ ]:

c1 = {"//div[@class='_2vDPL']" : {'cmd':'paste', 'text':'onami'}, 
      "//footer[@class='_3E8Fg']//p": {'cmd':'paste','text':'HI'}}

c2 = {msg_arrow('last()') : {'cmd':'paste', 'text':'onami'}}

In [ ]:
x.chain(c2)

### Develop WIP

In [ ]:
def sidepane(nodeno=2, panebase = "(//div[@id='pane-side']//div[@class='_3OvU8'])"):
    base = panebase + "[" + str(nodeno) + "]"
    panexpt = {'chat' : [base + "//div[1]/div[1]/span"],
            'lastmsg' : [base + "//div[1]/span/span[1]"],
            'msg_sender': [base + "//div[2]/div[1]/span/span[3]"],
            'dttm' : [base + "//div[@class='Dvjym']"]}
    panexpt['base'] = [base]
    return panexpt

sidepane_msg_xpt = {'chat' : ["//div[1]/div[1]/span"],'lastmsg' : ["//div[1]/span/span[1]"],
            'msg_sender': ["//div[2]/div[1]/span/span[3]"],'dttm' : ["//div[@class='Dvjym']"]}

In [ ]:
pane_container = "(//div[@id='pane-side']//div[@class='_8nE1Y'])"
pane_chat_name = (//div[@class='_8nE1Y'])[3]//div[@class='_21S-L']
last_text = (//div[@class='_8nE1Y'])[1]//div[@class='vQ0w7']/span[1]/span[3]
last_sender = (//div[@class='_8nE1Y'])[1]//div[@class='vQ0w7']/span[1]/span[1]
last_msg_time = ((//div[@class='_8nE1Y'])[1]/div[@class='y_sn4']/div[@class='Dvjym']
new_msg_notif = (//div[@class='_8nE1Y'])[1]//div[@class='Dvjym']//div[@class='_1pJ9J']/span

In [ ]:
msg_scrap(d,'last()')

In [ ]:
quoted_sender

In [ ]:
x.chatparse(,bodymsg)

In [ ]:
i = sidepane_scroll_by_id(d, initial = 1, scrolls=25)

In [ ]:
i = sidepane_scroll_by_id(d, initial = i, scrolls=25)

In [ ]:
select_chat(d, 'sm & cn')


In [ ]:
scroll_by_xpath(d, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]")

In [ ]:
scroll_by_xpath(d, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]")

In [ ]:

def iterdict(driver, xpd:dict, sec=3):
    rd = defaultdict()
    file = 'XPT_' + dt.datetime.now().strftime("%H%M %Y%m%d") + '.txt'
    fp = open(os.getcwd() + '\\output\\' + file, 'a')
    for k, v in xpd.items():
        rls = []
        for n in v:
            wait = WebDriverWait(driver, sec)
            try: rs = wait.until(EC.presence_of_element_located((By.XPATH, n))).text
            except: rs = False
            print(str(k) + chr(10) + str(n) + '::' + str(rs) + chr(10) + chr(10))
            fp.write(str(k) + chr(10) + str(n) + '::' + str(rs) + chr(10) + chr(10))
            rls.append(rs)
        rd = dict_update(rd, k, rls)
    else: 
        fp.close()
        return rd
            


In [ ]:

#xfill(d,"/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]","SM & CN_Monitoring")
#x.chatparse("(//div[@id='pane-side']//div[@class='_3OvU8'])",sidepane_xpt)


def iternode(driver, base_xpt, start_node, end_node, parsefn, scrollfn=None, ini=1, sc=25):
    wait = WebDriverWait(driver, 5)
    for n in range(start_node, end_node):
        x = base_xpt + '[' + str(n) + ']'
        try:
            el = wait.until(EC.presence_of_element_located((By.XPATH, x)))
        except:
            ini = scrollfn(driver, initial=ini, scrolls=sc)
            iternode(driver, base_xpt, start_node, end_node, scrollfn, ini, sc)
    else:
        print('data available')
        return 1

In [ ]:
msg_base = "(//div[@class='ItfyB _3nbHh'])"

In [ ]:
a = {'chat_name' : ["//span[@class='_3q9s6']", "//div[@class='zoWT4']}